<a href="https://colab.research.google.com/github/david132313/A_shareStock_data_and_model/blob/main/data_downloading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install tushare pandas pyarrow tqdm

from google.colab import drive
drive.mount("/content/drive")


In [ ]:
import os, time, json, random
import pandas as pd
import tushare as ts
from tqdm import tqdm
from time import perf_counter

# --- Token: env first, fallback to Colab Secrets ---
TS_TOKEN = os.environ.get("TUSHARE_TOKEN", "")
if not TS_TOKEN:
    try:
        from google.colab import userdata
        TS_TOKEN = userdata.get("E_TOKEN") or ""   # 这里要和你 Secrets 里 Name 完全一致
    except Exception:
        TS_TOKEN = ""

if not TS_TOKEN:
    raise RuntimeError(
        "Missing TuShare token.\n"
        "Option A (env): %env TUSHARE_TOKEN=YOUR_TOKEN\n"
        "Option B (Secrets): set a secret named E_TOKEN (or change the name in code)."
    )

ts.set_token(TS_TOKEN)
pro = ts.pro_api()

START = "20000101"
END   = "20091231"

BASE_DIR = "/content/drive/MyDrive/AshareDB_raw_2010_20251220"
DAILY_DIR = os.path.join(BASE_DIR, "daily_parquet")
META_DIR  = os.path.join(BASE_DIR, "meta")
os.makedirs(DAILY_DIR, exist_ok=True)
os.makedirs(META_DIR, exist_ok=True)

MANIFEST_PATH = os.path.join(META_DIR, "manifest_daily.csv")
FAIL_PATH     = os.path.join(META_DIR, "failures.json")
MISSING_PATH  = os.path.join(META_DIR, "missing_dates.csv")

# ✅ 不需要很频繁（多少天罗盘一次）
SAVE_EVERY_N_DAYS = 22

# ✅ 避免限频/抖动，可稍微 sleep
BASE_SLEEP = 0.05

def daily_path(d):
    return os.path.join(DAILY_DIR, f"daily_{d}.parquet")


In [ ]:
# 交易日历（开市日）
cal = pro.trade_cal(exchange="SSE", start_date=START, end_date=END, is_open="1")
trade_dates = cal["cal_date"].tolist()
cal.to_parquet(os.path.join(META_DIR, "trade_cal.parquet"), index=False)

# 已有文件 -> 视为已完成
have = set()
for fn in os.listdir(DAILY_DIR):
    if fn.startswith("daily_") and fn.endswith(".parquet") and len(fn) >= 6+8:
        have.add(fn[6:14])

need = set(trade_dates)
todo = sorted(list(need - have))

print("trade_days:", len(trade_dates))
print("already have:", len(have))
print("todo:", len(todo), "first:", (todo[0] if todo else None), "last:", (todo[-1] if todo else None))


trade_days: 2415
already have: 3878
todo: 2415 first: 20000104 last: 20091231


In [ ]:
def fetch_daily_one_day(trade_date: str) -> pd.DataFrame:
    return pro.daily(trade_date=trade_date)

# 读旧 manifest（断点续跑的“历史测速”也保留）
if os.path.exists(MANIFEST_PATH):
    manifest_df = pd.read_csv(MANIFEST_PATH)
    manifest_records = manifest_df.to_dict("records")
else:
    manifest_records = []

# 读旧 failures
if os.path.exists(FAIL_PATH):
    with open(FAIL_PATH, "r") as f:
        failures = json.load(f)
else:
    failures = []

def flush_state():
    pd.DataFrame(manifest_records).to_csv(MANIFEST_PATH, index=False)
    with open(FAIL_PATH, "w") as f:
        json.dump(failures, f, ensure_ascii=False, indent=2)

def safe_fetch_and_save(d: str, max_retry=6):
    out = daily_path(d)
    if os.path.exists(out):
        return {"date": d, "status": "skip_exists", "rows": None,
                "api_s": 0.0, "write_s": 0.0, "total_s": 0.0}

    t_total0 = perf_counter()
    last_err = None

    for k in range(max_retry):
        try:
            # API 计时
            t0 = perf_counter()
            df = fetch_daily_one_day(d)
            api_s = perf_counter() - t0

            # 写文件计时
            t1 = perf_counter()
            df.to_parquet(out, index=False)
            write_s = perf_counter() - t1

            total_s = perf_counter() - t_total0
            return {"date": d, "status": "ok", "rows": int(len(df)),
                    "api_s": api_s, "write_s": write_s, "total_s": total_s}

        except Exception as e:
            last_err = repr(e)
            # 指数退避 + 抖动
            sleep_s = min(60, (2 ** k)) + random.random()
            time.sleep(sleep_s)

    total_s = perf_counter() - t_total0
    return {"date": d, "status": "fail", "rows": None,
            "api_s": None, "write_s": None, "total_s": total_s, "err": last_err}


In [ ]:
new_fail_map = {}  # 用于去重 failures（按 date）

# 把历史 failures 做成 map，避免重复堆积
for item in failures:
    new_fail_map[item["date"]] = item

n_done = 0
for i, d in enumerate(tqdm(todo, desc="download daily")):
    r = safe_fetch_and_save(d)
    manifest_records.append(r)

    if r["status"] == "fail":
        new_fail_map[d] = r  # 覆盖更新最新错误信息

    n_done += 1

    # ✅ 每 SAVE_EVERY_N_DAYS 天落盘一次（约每月）
    if (n_done % SAVE_EVERY_N_DAYS) == 0:
        failures = list(new_fail_map.values())
        flush_state()
        print(f"\n[checkpoint] saved state at {d} | total_records={len(manifest_records)} | fails={len(failures)}")

    time.sleep(BASE_SLEEP)

# 最终落盘
failures = list(new_fail_map.values())
flush_state()

print("Finished run. Total manifest records:", len(manifest_records), "failures:", len(failures))


In [ ]:
import glob

files = glob.glob(os.path.join(DAILY_DIR, "daily_*.parquet"))
have2 = set(os.path.basename(x)[6:14] for x in files)
need2 = set(trade_dates)

missing = sorted(list(need2 - have2))
extra   = sorted(list(have2 - need2))

pd.DataFrame({"missing_trade_date": missing}).to_csv(MISSING_PATH, index=False)

print("need:", len(need2), "have:", len(have2))
print("missing:", len(missing), "extra:", len(extra))
print("first 20 missing:", missing[:20])
print("missing list saved to:", MISSING_PATH)


need: 2415 have: 6293
missing: 0 extra: 3878
first 20 missing: []
missing list saved to: /content/drive/MyDrive/AshareDB_raw_2010_20251220/meta/missing_dates.csv


In [ ]:
df = pd.read_csv(MANIFEST_PATH)

ok = df[df["status"] == "ok"].copy()
print("ok days:", len(ok), "fail days:", (df["status"]=="fail").sum(), "skip:", (df["status"]=="skip_exists").sum())

if len(ok) > 0:
    print("\n=== time stats (seconds) ===")
    print(ok[["api_s","write_s","total_s"]].describe(percentiles=[0.5,0.9,0.95,0.99]))

    # 慢的日期 Top 20
    slow = ok.sort_values("total_s", ascending=False).head(20)
    print("\n=== slowest 20 days ===")
    print(slow[["date","rows","api_s","write_s","total_s"]].to_string(index=False))

    # 估算吞吐：rows/秒（仅 rough）
    ok["rows_per_s"] = ok["rows"] / ok["total_s"]
    print("\n=== throughput rows/s (rough) ===")
    print(ok["rows_per_s"].describe(percentiles=[0.5,0.9,0.95]))


ok days: 6293 fail days: 0 skip: 0

=== time stats (seconds) ===
             api_s      write_s      total_s
count  6293.000000  6293.000000  6293.000000
mean      1.703361     0.029579     1.888490
std       1.477106     0.007304     3.122325
min       0.926046     0.017360     0.948526
50%       1.465908     0.027523     1.495149
90%       2.502278     0.039582     2.551514
95%       2.992445     0.045079     3.052770
99%       5.177768     0.053931     6.695394
max      34.218976     0.127822   102.355856

=== slowest 20 days ===
    date  rows     api_s  write_s    total_s
20130801  2361 11.639437 0.026518 102.355856
20200511  3890  2.395036 0.029543  77.140523
20180614  3332  9.039452 0.025601  54.524611
20140715  2303  6.758948 0.024018  47.079788
20140226  2371 23.384493 0.030529  46.549495
20210517  4377 12.137307 0.030899  45.423148
20230913  5273  8.829159 0.028806  44.687206
20190508  3611  2.155453 0.027419  43.081656
20160526  2597  9.193765 0.025907  41.619552
20180918  

In [ ]:
!pip -q install pandas pyarrow tqdm

from google.colab import drive
drive.mount("/content/drive")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os, re, time, json, shutil, hashlib, sqlite3
import pandas as pd
from tqdm import tqdm
from time import perf_counter
from datetime import datetime

# 你之前下载的 raw 数据根目录（保持一致）
BASE_DIR = "/content/drive/MyDrive/AshareDB_raw_2010_20251220"
DAILY_DIR = os.path.join(BASE_DIR, "daily_parquet")
META_DIR  = os.path.join(BASE_DIR, "meta")

# Drive 上数据库输出位置
DB_DIR_DRIVE = os.path.join(BASE_DIR, "db")
BK_DIR_DRIVE = os.path.join(BASE_DIR, "backups")

os.makedirs(DB_DIR_DRIVE, exist_ok=True)
os.makedirs(BK_DIR_DRIVE, exist_ok=True)

DB_DRIVE_PATH = os.path.join(DB_DIR_DRIVE, "ashare.sqlite")

# Colab 本地构建位置（快 + 稳）
LOCAL_WORKDIR = "/content/ashare_db_build"
os.makedirs(LOCAL_WORKDIR, exist_ok=True)
DB_LOCAL_PATH = os.path.join(LOCAL_WORKDIR, "ashare.sqlite")

print("DAILY_DIR:", DAILY_DIR)
print("DB_LOCAL_PATH:", DB_LOCAL_PATH)
print("DB_DRIVE_PATH:", DB_DRIVE_PATH)


DAILY_DIR: /content/drive/MyDrive/AshareDB_raw_2010_20251220/daily_parquet
DB_LOCAL_PATH: /content/ashare_db_build/ashare.sqlite
DB_DRIVE_PATH: /content/drive/MyDrive/AshareDB_raw_2010_20251220/db/ashare.sqlite


In [ ]:
def atomic_copy(src: str, dst: str):
    """先复制到临时文件，再 replace，避免 Drive 上出现半写入坏库。"""
    os.makedirs(os.path.dirname(dst), exist_ok=True)
    tmp = dst + ".tmp"
    shutil.copy2(src, tmp)
    os.replace(tmp, dst)

def backup_drive_db(dst_db_path: str, backups_dir: str):
    """把 Drive 上现有 db 做一个版本备份（如果存在）。"""
    if os.path.exists(dst_db_path):
        ts = datetime.now().strftime("%Y%m%d_%H%M%S")
        bk = os.path.join(backups_dir, f"ashare_{ts}.sqlite")
        shutil.copy2(dst_db_path, bk)
        return bk
    return None


In [ ]:
def connect_sqlite(db_path: str) -> sqlite3.Connection:
    conn = sqlite3.connect(db_path, timeout=60)
    conn.isolation_level = None   # ✅ 关键：关闭隐式事务，允许我们手动 BEGIN/COMMIT
    conn.execute("PRAGMA foreign_keys=ON;")
    conn.execute("PRAGMA journal_mode=WAL;")
    conn.execute("PRAGMA synchronous=NORMAL;")
    conn.execute("PRAGMA temp_store=MEMORY;")
    conn.execute("PRAGMA cache_size=-200000;")
    conn.execute("PRAGMA busy_timeout=60000;")
    return conn


SCHEMA_SQL = r"""
CREATE TABLE IF NOT EXISTS security_master (
  ts_code     TEXT PRIMARY KEY,
  symbol      TEXT,
  name        TEXT,
  area        TEXT,
  industry    TEXT,
  market      TEXT,
  exchange    TEXT,
  list_date   TEXT,
  delist_date TEXT,
  updated_at  TEXT
);

CREATE TABLE IF NOT EXISTS trade_calendar (
  trade_date TEXT PRIMARY KEY,   -- YYYYMMDD
  exchange   TEXT,
  is_open    INTEGER
);

CREATE TABLE IF NOT EXISTS daily_price (
  trade_date TEXT NOT NULL,
  ts_code    TEXT NOT NULL,
  open       REAL,
  high       REAL,
  low        REAL,
  close      REAL,
  pre_close  REAL,
  change     REAL,
  pct_chg    REAL,
  vol        REAL,
  amount     REAL,
  PRIMARY KEY (trade_date, ts_code),
  FOREIGN KEY (ts_code) REFERENCES security_master(ts_code)
);

CREATE INDEX IF NOT EXISTS idx_daily_ts_date ON daily_price(ts_code, trade_date);
CREATE INDEX IF NOT EXISTS idx_daily_date ON daily_price(trade_date);

-- 断点/文件清单：记录哪些交易日已经入库
CREATE TABLE IF NOT EXISTS daily_file_manifest (
  trade_date TEXT PRIMARY KEY,
  parquet_file TEXT NOT NULL,
  file_mtime REAL NOT NULL,
  file_size  INTEGER NOT NULL,
  rows       INTEGER NOT NULL,
  loaded_at  TEXT NOT NULL
);

-- 每日入库计时日志
CREATE TABLE IF NOT EXISTS etl_day_log (
  trade_date TEXT PRIMARY KEY,
  parquet_file TEXT NOT NULL,
  status TEXT NOT NULL,              -- ok/skip/fail
  rows INTEGER,
  read_s REAL,
  insert_s REAL,
  total_s REAL,
  message TEXT,
  logged_at TEXT NOT NULL
);

-- 每次批处理总日志
CREATE TABLE IF NOT EXISTS etl_run_log (
  run_id INTEGER PRIMARY KEY AUTOINCREMENT,
  started_at TEXT NOT NULL,
  ended_at   TEXT,
  status     TEXT,
  message    TEXT,
  days_total INTEGER,
  days_ok    INTEGER,
  days_skip  INTEGER,
  days_fail  INTEGER
);

-- 防误删：禁止 DELETE（避免手滑删表/删行）
CREATE TRIGGER IF NOT EXISTS trg_no_delete_daily
BEFORE DELETE ON daily_price
BEGIN
  SELECT RAISE(ABORT, 'DELETE is disabled on daily_price');
END;

CREATE TRIGGER IF NOT EXISTS trg_no_delete_sec
BEFORE DELETE ON security_master
BEGIN
  SELECT RAISE(ABORT, 'DELETE is disabled on security_master');
END;
"""

UPSERT_DAILY_SQL = """
INSERT INTO daily_price (
  trade_date, ts_code, open, high, low, close, pre_close, change, pct_chg, vol, amount
) VALUES (?,?,?,?,?,?,?,?,?,?,?)
ON CONFLICT(trade_date, ts_code) DO UPDATE SET
  open=excluded.open,
  high=excluded.high,
  low=excluded.low,
  close=excluded.close,
  pre_close=excluded.pre_close,
  change=excluded.change,
  pct_chg=excluded.pct_chg,
  vol=excluded.vol,
  amount=excluded.amount;
"""

UPSERT_SEC_SQL = """
INSERT INTO security_master (
  ts_code, symbol, name, area, industry, market, exchange, list_date, delist_date, updated_at
) VALUES (?,?,?,?,?,?,?,?,?,?)
ON CONFLICT(ts_code) DO UPDATE SET
  symbol=excluded.symbol,
  name=excluded.name,
  area=excluded.area,
  industry=excluded.industry,
  market=excluded.market,
  exchange=excluded.exchange,
  list_date=excluded.list_date,
  delist_date=excluded.delist_date,
  updated_at=excluded.updated_at;
"""
INSERT_SEC_MIN_SQL = """
INSERT INTO security_master(ts_code, updated_at)
VALUES (?, ?)
ON CONFLICT(ts_code) DO UPDATE SET
  updated_at=excluded.updated_at;
"""

def init_db(conn: sqlite3.Connection):
    conn.executescript(SCHEMA_SQL)
    conn.commit()

# 如果 Drive 上已有 db，可以先拉到本地继续增量；否则从头建
if os.path.exists(DB_DRIVE_PATH):
    print("Found DB on Drive -> copying to local work db...")
    shutil.copy2(DB_DRIVE_PATH, DB_LOCAL_PATH)
else:
    print("No DB on Drive -> creating new local db...")
    if os.path.exists(DB_LOCAL_PATH):
        os.remove(DB_LOCAL_PATH)

conn = connect_sqlite(DB_LOCAL_PATH)
init_db(conn)

print("DB ready:", DB_LOCAL_PATH)


Found DB on Drive -> copying to local work db...
DB ready: /content/ashare_db_build/ashare.sqlite


In [ ]:
stock_basic_path = os.path.join(META_DIR, "stock_basic.parquet")
if os.path.exists(stock_basic_path):
    sb = pd.read_parquet(stock_basic_path)
    # 统一字段（你保存时用的字段是：ts_code,symbol,name,area,industry,market,list_date）
    # exchange/delist_date 可能没有，留空
    now = datetime.now().isoformat(timespec="seconds")
    rows = []
    for _, r in sb.iterrows():
        rows.append((
            r.get("ts_code"),
            r.get("symbol"),
            r.get("name"),
            r.get("area"),
            r.get("industry"),
            r.get("market"),
            None,                # exchange
            r.get("list_date"),
            None,                # delist_date
            now
        ))
    conn.executemany(UPSERT_SEC_SQL, rows)
    conn.commit()
    print("security_master upserted rows:", len(rows))
else:
    print("No stock_basic.parquet found (skip security_master load).")


No stock_basic.parquet found (skip security_master load).


In [ ]:
DATE_RE = re.compile(r"^daily_(\d{8})\.parquet$")

all_files = []
for fn in os.listdir(DAILY_DIR):
    m = DATE_RE.match(fn)
    if m:
        all_files.append((m.group(1), os.path.join(DAILY_DIR, fn)))

all_files.sort(key=lambda x: x[0])
print("daily parquet files:", len(all_files), "first:", all_files[0][0] if all_files else None, "last:", all_files[-1][0] if all_files else None)

# 已入库日期
loaded = set(r[0] for r in conn.execute("SELECT trade_date FROM daily_file_manifest").fetchall())
todo = [(d,p) for (d,p) in all_files if d not in loaded]

print("already loaded:", len(loaded))
print("todo:", len(todo))


daily parquet files: 3878 first: 20100104 last: 20251219
already loaded: 0
todo: 3878


In [ ]:
SAVE_EVERY_N_DAYS = 22   # 约每月落盘一次
MAX_FAIL_KEEP = 5000     # fail log 上限（避免无限增长）

def file_stat(path: str):
    st = os.stat(path)
    return st.st_mtime, st.st_size

def log_day(conn, trade_date, parquet_file, status, rows=None, read_s=None, insert_s=None, total_s=None, message=None):
    conn.execute("""
    INSERT INTO etl_day_log(trade_date, parquet_file, status, rows, read_s, insert_s, total_s, message, logged_at)
    VALUES (?,?,?,?,?,?,?,?,?)
    ON CONFLICT(trade_date) DO UPDATE SET
      parquet_file=excluded.parquet_file,
      status=excluded.status,
      rows=excluded.rows,
      read_s=excluded.read_s,
      insert_s=excluded.insert_s,
      total_s=excluded.total_s,
      message=excluded.message,
      logged_at=excluded.logged_at;
    """, (
        trade_date, parquet_file, status, rows, read_s, insert_s, total_s, message,
        datetime.now().isoformat(timespec="seconds")
    ))

def mark_manifest(conn, trade_date, parquet_file, mtime, size, rows):
    conn.execute("""
    INSERT INTO daily_file_manifest(trade_date, parquet_file, file_mtime, file_size, rows, loaded_at)
    VALUES (?,?,?,?,?,?)
    ON CONFLICT(trade_date) DO UPDATE SET
      parquet_file=excluded.parquet_file,
      file_mtime=excluded.file_mtime,
      file_size=excluded.file_size,
      rows=excluded.rows,
      loaded_at=excluded.loaded_at;
    """, (
        trade_date, parquet_file, float(mtime), int(size), int(rows),
        datetime.now().isoformat(timespec="seconds")
    ))

def integrity_ok(conn) -> bool:
    try:
        r = conn.execute("PRAGMA integrity_check;").fetchone()[0]
        return (r == "ok")
    except Exception:
        return False

# 记录一次 run
run_start = datetime.now().isoformat(timespec="seconds")
cur = conn.execute("INSERT INTO etl_run_log(started_at, status, message, days_total) VALUES (?,?,?,?)",
                   (run_start, "running", "ingest daily parquet", len(todo)))
run_id = cur.lastrowid
conn.commit()

ok_n = skip_n = fail_n = 0

# Drive 上已有 db 先备份一份（可选，但强烈推荐）
bk = backup_drive_db(DB_DRIVE_PATH, BK_DIR_DRIVE)
if bk:
    print("Drive DB backed up to:", bk)

for i, (d, p) in enumerate(tqdm(todo, desc="ingest daily_parquet")):
    t_total0 = perf_counter()
    try:
        # 读 parquet 计时
        t0 = perf_counter()
        df = pd.read_parquet(p)
        read_s = perf_counter() - t0

        # 提取字段并准备 tuples（只保留日线核心量价字段）
        # tushare daily 默认字段：ts_code, trade_date, open, high, low, close, pre_close, change, pct_chg, vol, amount
        cols = ["trade_date","ts_code","open","high","low","close","pre_close","change","pct_chg","vol","amount"]
        missing_cols = [c for c in cols if c not in df.columns]
        if missing_cols:
            raise ValueError(f"Missing columns in {os.path.basename(p)}: {missing_cols}")

        # 转成 python tuple list（NaN -> None）
        sub = df[cols].where(pd.notnull(df[cols]), None)
        rows = [tuple(x) for x in sub.to_numpy()]
        nrows = len(rows)

        # 插入计时（每个交易日一笔事务，断点安全）
        t1 = perf_counter()
        conn.execute("BEGIN;")
        conn.executemany(UPSERT_DAILY_SQL, rows)

        # 更新 manifest（记录本文件已入库）
        mtime, size = file_stat(p)
        mark_manifest(conn, d, os.path.basename(p), mtime, size, nrows)

        conn.commit()
        insert_s = perf_counter() - t1

        total_s = perf_counter() - t_total0
        log_day(conn, d, os.path.basename(p), "ok", rows=nrows, read_s=read_s, insert_s=insert_s, total_s=total_s)

        ok_n += 1

    except Exception as e:
        conn.rollback()
        total_s = perf_counter() - t_total0
        log_day(conn, d, os.path.basename(p), "fail", message=repr(e), total_s=total_s)
        fail_n += 1

    # checkpoint：约每月一次保存到 Drive（避免频繁写 Drive）
    if (ok_n + fail_n) % SAVE_EVERY_N_DAYS == 0:
        conn.commit()
        # 自检
        ok = integrity_ok(conn)
        if not ok:
            raise RuntimeError("Integrity check failed during checkpoint")
        # 保存到 Drive（原子覆盖）
        atomic_copy(DB_LOCAL_PATH, DB_DRIVE_PATH)
        print(f"\n[checkpoint] saved DB to Drive at {d} | ok={ok_n} fail={fail_n}")

# 最终保存一次
conn.commit()
ok = integrity_ok(conn)
if not ok:
    raise RuntimeError("Integrity check failed at final save")
atomic_copy(DB_LOCAL_PATH, DB_DRIVE_PATH)

run_end = datetime.now().isoformat(timespec="seconds")
conn.execute("""
UPDATE etl_run_log
SET ended_at=?, status=?, message=?, days_ok=?, days_skip=?, days_fail=?
WHERE run_id=?
""", (run_end, "finished", "ingest completed", ok_n, skip_n, fail_n, run_id))
conn.commit()

print("DONE. ok:", ok_n, "fail:", fail_n)
print("Saved DB to Drive:", DB_DRIVE_PATH)


In [ ]:
import pandas as pd, sqlite3

conn = sqlite3.connect("/content/ashare_db_build/ashare.sqlite")
df_fail = pd.read_sql_query("""
SELECT trade_date, message
FROM etl_day_log
WHERE status='fail'
ORDER BY trade_date
LIMIT 10;
""", conn)
df_fail


,trade_date,message
0,20100104,IntegrityError('FOREIGN KEY constraint failed')
1,20100105,IntegrityError('FOREIGN KEY constraint failed')
2,20100106,IntegrityError('FOREIGN KEY constraint failed')
3,20100107,IntegrityError('FOREIGN KEY constraint failed')
4,20100108,IntegrityError('FOREIGN KEY constraint failed')
5,20100111,IntegrityError('FOREIGN KEY constraint failed')
6,20100112,IntegrityError('FOREIGN KEY constraint failed')
7,20100113,IntegrityError('FOREIGN KEY constraint failed')
8,20100114,IntegrityError('FOREIGN KEY constraint failed')
9,20100115,IntegrityError('FOREIGN KEY constraint failed')
